Using the natural gas data, this code calculates the contract value given multiple costs involved in storing natural gas like injection cost, withdrawal cost, storage cost, etc. The code can take multiple injection and withdrawal dates as input. The code uses upsampled historical data with 1-year price prediction.

In [1]:
# Importing necessary libraries for data handling and visualization
import numpy as np
import pandas as pd
from datetime import date

In [2]:
# Load the data
data = pd.read_csv('UpsampledData.csv')  # Load the dataset with upsampled data

In [3]:
# Define a function to calculate the net profit from storage contracts
def calculate_contract_value(injection_dates, withdrawal_dates, volume):
    """
    Calculate the net profit from commodity trading with storage costs.
    
    Parameters:
    injection_dates : list of date
        Dates when commodities are injected into storage.
    withdrawal_dates : list of date
        Dates when commodities are withdrawn from storage.
    volume : int
        Volume of commodities traded in MMBtu.
    
    Returns:
    float
        Net profit from the trading contract.
    """
    
    # Fixed cost parameters
    storage_cost_per_month = 100_000  # Cost per month for storage
    injection_withdrawal_cost = 10_000  # Cost for each injection or withdrawal
    transport_cost = 50_000  # Transportation cost

    # Initialize total calculations
    total_storage_duration = 0  # Total storage duration in months
    total_injection_withdrawal_cost = injection_withdrawal_cost * (len(injection_dates) + len(withdrawal_dates))
    total_buy_cost = 0
    total_sell_revenue = 0

    # Process each injection and withdrawal date
    for i in range(len(injection_dates)):
        in_date = injection_dates[i]
        out_date = withdrawal_dates[i]
        total_storage_duration += (out_date.year - in_date.year) * 12 + (out_date.month - in_date.month)
        
        # Fetching buying and selling prices from the dataset
        buy_price = data[data['Dates'] == str(in_date)]['Prices'].values[-1]
        total_buy_cost += buy_price * volume
        print(f"Price on {in_date}: {buy_price}")
        
        sell_price = data[data['Dates'] == str(out_date)]['Prices'].values[-1]
        total_sell_revenue += sell_price * volume
        print(f"Price on {out_date}: {sell_price}")
        
    # Calculate total costs and revenue
    total_storage_cost = storage_cost_per_month * total_storage_duration
    total_injection_withdrawal_cost *= (volume / 1_000_000)  # Adjust cost per million MMBtu
    total_transport_cost = transport_cost * 2  # Transport to and from the location

    # Calculate net profit
    net_profit = total_sell_revenue - total_buy_cost - total_storage_cost - total_injection_withdrawal_cost - total_transport_cost

    return np.round(net_profit, 2)

In [4]:
print(calculate_contract_value(
    injection_dates=[date(2022, 1, 1), date(2022, 2, 1), date(2022, 2, 21), date(2022, 4, 1)], 
    withdrawal_dates=[date(2022, 1, 27), date(2022, 2, 15), date(2022, 3, 20), date(2022, 6, 1)], 
    volume=1_000_000
))

Price on 2022-01-01: 11.400740046452624
Price on 2022-01-27: 11.468350319248154
Price on 2022-02-01: 11.50906669738608
Price on 2022-02-15: 11.683825030463495
Price on 2022-02-21: 11.755261369958324
Price on 2022-03-20: 11.69888219463267
Price on 2022-04-01: 11.476909093432209
Price on 2022-06-01: 10.695717987428402
-1075201.68
